In [5]:
!pip install pypdf langchain_community langchain-text-splitters langchain sentence_transformers langchain-chroma -q

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("/workspaces/ask-cv/IT Support .pdf")
pages = loader.load_and_split()

In [7]:
pages

[Document(page_content="Question:\nWhy\nis\nmy\nlaptop\nrunning\nslow?\nShort\nSolution:\nThere\nare\nseveral\nreasons\nfor\na\nslow\nlaptop.\nCheck\nif\nmany\nprograms\nare\nrunning\nat\nonce,\nfree\nup\ndisk\nspace\nby\ndeleting\nunnecessary\nfiles,\nconsider\nupgrading\nRAM,\nor\nrun\na\nscan\nfor\nmalware.\nQuestion:\nHow\ndo\nI\nconnect\nan\nexternal\nhard\ndrive\nto\nmy\ncomputer?\nShort\nSolution:\nMost\nexternal\nhard\ndrives\nconnect\nvia\nUSB.\nLocate\na\nfree\nUSB\nport\non\nyour\ncomputer,\nplug\nin\nthe\ndrive,\nand\nit\nshould\nbe\nautomatically\ndetected\nby\nyour\noperating\nsystem.\nQuestion:\nWhat\nare\nthe\nsystem\nrequirements\nfor\nrunning\na\nspecific\nsoftware?\nShort\nSolution:\nSystem\nrequirements\ncan\nusually\nbe\nfound\non\nthe\nsoftware\ndeveloper's\nwebsite\nor\nthe\nsoftware\npackaging\nitself.\nThey\nspecify\nthe\nminimum\nRAM,\nstorage\nspace,\nprocessor\ntype,\nand\noperating\nsystem\nneeded\nto\nrun\nthe\nsoftware\nsmoothly.\nSoftware-related\nQuesti

In [8]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [9]:
from langchain_community.vectorstores import FAISS

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/codespace/.python/current/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
!pip install faiss-cpu

In [12]:
db = FAISS.from_documents(texts, embeddings)

In [13]:
print(db.index.ntotal)

7


In [14]:
query = "can solve linear advection/diffusion problems"
docs = db.similarity_search(query)

In [15]:
docs

[Document(page_content='Short\nSolution:\nBacking\nup\ndata\ninvolves\ncopying\nit\nto\nanother\nstorage\nlocation\n(external\nhard\ndrive,\ncloud\nstorage).\nMany\nbackup\noptions\nexist\ndepending\non\nyour\nneeds.\nRestoring\ndata\ninvolves\nretrieving\nit\nfrom\nyour\nbackup\nstorage.\nQuestion:\nCan\nyou\nrecommend\na\ngood\nantivirus\nsoftware?\nShort\nSolution:\nSeveral\nreputable\nantivirus\nprograms\nare\navailable.\nPopular\noptions\ninclude\nBitdefender,\nNorton,\nand\nKaspersky.\nResearch\nand\nchoose\none\nthat\nsuits\nyour\nneeds\nand\nbudget.\nQuestion:\nWhy\nis\nmy\nlaptop\nrunning\nslow?\nShort\nSolution:\nThere\nare\nseveral\nreasons\nfor\na\nslow\nlaptop.\nCheck\nif\nmany\nprograms\nare\nrunning\nat\nonce,\nfree\nup\ndisk\nspace\nby\ndeleting\nunnecessary\nfiles,\nconsider\nupgrading\nRAM,\nor\nrun\na\nscan\nfor\nmalware.\nQuestion:\nHow\ndo\nI\nconnect\nan\nexternal\nhard\ndrive\nto\nmy\ncomputer?\nShort\nSolution:\nMost\nexternal\nhard\ndrives\nconnect\nvia\nUSB.\n

In [16]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

In [17]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores 

[(Document(page_content='Short\nSolution:\nBacking\nup\ndata\ninvolves\ncopying\nit\nto\nanother\nstorage\nlocation\n(external\nhard\ndrive,\ncloud\nstorage).\nMany\nbackup\noptions\nexist\ndepending\non\nyour\nneeds.\nRestoring\ndata\ninvolves\nretrieving\nit\nfrom\nyour\nbackup\nstorage.\nQuestion:\nCan\nyou\nrecommend\na\ngood\nantivirus\nsoftware?\nShort\nSolution:\nSeveral\nreputable\nantivirus\nprograms\nare\navailable.\nPopular\noptions\ninclude\nBitdefender,\nNorton,\nand\nKaspersky.\nResearch\nand\nchoose\none\nthat\nsuits\nyour\nneeds\nand\nbudget.\nQuestion:\nWhy\nis\nmy\nlaptop\nrunning\nslow?\nShort\nSolution:\nThere\nare\nseveral\nreasons\nfor\na\nslow\nlaptop.\nCheck\nif\nmany\nprograms\nare\nrunning\nat\nonce,\nfree\nup\ndisk\nspace\nby\ndeleting\nunnecessary\nfiles,\nconsider\nupgrading\nRAM,\nor\nrun\na\nscan\nfor\nmalware.\nQuestion:\nHow\ndo\nI\nconnect\nan\nexternal\nhard\ndrive\nto\nmy\ncomputer?\nShort\nSolution:\nMost\nexternal\nhard\ndrives\nconnect\nvia\nUSB.\

In [18]:
db.save_local("faiss_index")

new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [19]:
docs = retriever.invoke("what is cavity")

In [20]:
docs 

[Document(page_content='process.\nQuestion:\nCan\nyou\nguide\nme\nthrough\nthe\nprocess\nof\nuninstalling\na\nprogram?\nShort\nSolution:\nThe\nuninstall\nprocess\nvaries\nslightly\ndepending\non\nthe\noperating\nsystem.\nGenerally,\nyou\ncan\nuninstall\nprograms\nthrough\nthe\ncontrol\npanel\nor\nsettings\nmenu\non\nyour\ncomputer.\nNetwork\nand\nInternet-related\nQuestions:\nQuestion:\nHow\ndo\nI\nconnect\nto\na\nWi-Fi\nnetwork?\nShort\nSolution:\nLocate\nthe\nWi-Fi\nicon\non\nyour\ndevice\n(computer,\nphone,\netc.).\nSearch\nfor\navailable\nnetworks,\nselect\nyour\ndesired\nnetwork,\nand\nenter\nthe\npassword\n(if\nrequired).', metadata={'source': '/workspaces/ask-cv/IT Support .pdf', 'page': 1}),
 Document(page_content="software\nsmoothly.\nSoftware-related\nQuestions:\nQuestion:\nHow\ndo\nI\ninstall\na\nnew\nsoftware\napplication?\nShort\nSolution:\nDownloaded\nsoftware\noften\ncomes\nas\nan\ninstaller\nfile\n(e.g.,\n.exe).\nDouble-click\nthe\nfile\nand\nfollow\nthe\non-screen\nins